<h1>目录<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#TF-IDF" data-toc-modified-id="TF-IDF-1">TF-IDF</a></span></li><li><span><a href="#TextRank" data-toc-modified-id="TextRank-2">TextRank</a></span></li><li><span><a href="#导包" data-toc-modified-id="导包-3">导包</a></span></li><li><span><a href="#数据准备" data-toc-modified-id="数据准备-4">数据准备</a></span></li><li><span><a href="#提取关键词" data-toc-modified-id="提取关键词-5">提取关键词</a></span><ul class="toc-item"><li><span><a href="#TF-IDF方法" data-toc-modified-id="TF-IDF方法-5.1">TF-IDF方法</a></span></li><li><span><a href="#实验结果" data-toc-modified-id="实验结果-5.2">实验结果</a></span></li><li><span><a href="#TextRank方法" data-toc-modified-id="TextRank方法-5.3">TextRank方法</a></span></li><li><span><a href="#实验结果" data-toc-modified-id="实验结果-5.4">实验结果</a></span></li></ul></li><li><span><a href="#自动摘要" data-toc-modified-id="自动摘要-6">自动摘要</a></span><ul class="toc-item"><li><span><a href="#数据准备" data-toc-modified-id="数据准备-6.1">数据准备</a></span></li><li><span><a href="#TextRank方法" data-toc-modified-id="TextRank方法-6.2">TextRank方法</a></span></li></ul></li></ul></div>

# TF-IDF

常用于评估在一个文档集中一个词对某份文档的重要程度

TF:统计一个词在一篇文档中出现的频次

IDF:统计一个词在文档集中的多少个文档出现，反映了这个词对文档的区分能力

# TextRank

核心思想来源于PageRank

主要特色：脱离Corpus的支持，仅对单篇文档分析就可以提取该文档的关键词

# 导包

In [20]:
import math
import jieba
import jieba.posseg as psg
from gensim import corpora, models
from jieba import analyse
import functools
from summa import summarizer

# 数据准备

In [2]:
import json

path = './assignment_data/article_exporter.json'
with open(path, 'r', encoding='utf8', errors='ignore') as file:   
    data = json.load(file)

In [3]:
data[:2]

[{'stock_name': '浦发银行',
  'title': '目前这个价格还是很合适买入持有。12.75',
  'author': '小秘书',
  'timing': '11-11 15:03',
  'content': '  在12.5元至13元区间，还是值得投入长持。有一些前提一定要明确的，一是浦发的情况已经改善，还会继续完备经营管理向好。二来是市场中的浦发走向已转向右侧交易的方向，与几年前已不一样。从外资进来增持，从股市机制变化，从完备做空机制等等都有利于资金进一步加入浦发阵营。浦发今年的回归合理估值慢于好几个银行，这种情况将于现在起的一段时间会赶上。浦发的第三季业绩已公布。现阶段进入浦发，已没有几年前强弓之末时的失落。好好珍惜这时光。（从浦发明确15.05元已是最低可接受的转股价来看，向上的空间还是不少，虽然不是即时而要一些时间，但浦发已给你确立明确的信号。）2020年上证到2600点以下的观点不变，并不影响我对浦发的驻守。                                      '},
 {'stock_name': '浦发银行',
  'title': '浦发一发完债，就跌跌不休，有点提起裤子，就不认账的赶脚[大笑]',
  'author': 'Su999',
  'timing': '11-11 14:05',
  'content': '  浦发一发完债，就跌跌不休，有点提起裤子，就不认账的赶脚                                      '}]

In [5]:
def get_stop_words(path):
    """从路径path中加载停止词词典"""
    with open(path, encoding='utf-8') as file:
        return [line.strip() for line in file]

stop_words = get_stop_words('./data/stop_words.utf8')

docs_list = []
for doc in data:
    split_words = list(jieba.cut(doc['content'].strip()))
    split_words = [word for word in split_words if word not in stop_words and word != " "]
    if split_words:
        docs_list.append(split_words)
docs_list[:2]

[['12.5',
  '元至',
  '13',
  '元',
  '区间',
  '值得',
  '投入',
  '长持',
  '前提',
  '一定',
  '明确',
  '一是',
  '浦发',
  '情况',
  '已经',
  '改善',
  '还会',
  '继续',
  '完备',
  '经营',
  '管理',
  '市场',
  '中',
  '浦发',
  '走向',
  '转向',
  '右侧',
  '交易',
  '方向',
  '几年',
  '前',
  '外资',
  '进来',
  '增持',
  '股市',
  '机制',
  '变化',
  '完备',
  '做空',
  '机制',
  '有利于',
  '资金',
  '进一步',
  '加入',
  '浦发',
  '阵营',
  '浦发',
  '今年',
  '回归',
  '合理',
  '估值',
  '慢于',
  '好几个',
  '银行',
  '这种',
  '情况',
  '现在',
  '一段时间',
  '会',
  '赶上',
  '浦发',
  '第三季',
  '业绩',
  '公布',
  '现阶段',
  '进入',
  '浦发',
  '没有',
  '几年',
  '前强',
  '弓之末',
  '时',
  '失落',
  '好好',
  '珍惜',
  '时光',
  '浦发',
  '明确',
  '15.05',
  '元',
  '最低',
  '接受',
  '转',
  '股价',
  '来看',
  '向上',
  '空间',
  '不少',
  '即时',
  '时间',
  '浦发',
  '确立',
  '明确',
  '信号',
  '2020',
  '年',
  '上证',
  '2600',
  '点',
  '以下',
  '观点',
  '不变',
  '影响',
  '浦发',
  '驻守'],
 ['浦发', '一发', '完债', '跌跌', '不休', '有点', '提起', '裤子', '不认账', '赶脚']]

# 提取关键词

## TF-IDF方法

In [6]:
# idf值统计方法
def train_idf(doc_list):
    idf_dic = {}
    # 总文档数
    tt_count = len(doc_list)

    # 每个词出现的文档数
    for doc in doc_list:
        for word in set(doc):
            idf_dic[word] = idf_dic.get(word, 0.0) + 1.0

    # 按公式转换为idf值，分母加1进行平滑处理
    for k, v in idf_dic.items():
        idf_dic[k] = math.log(tt_count / (1.0 + v))

    # 对于没有在字典中的词，默认其仅在一个文档出现，得到默认idf值
    default_idf = math.log(tt_count / (1.0))
    return idf_dic, default_idf

In [7]:
#  排序函数，用于topK关键词的按值排序
def cmp(e1, e2):
    import numpy as np
    res = np.sign(e1[1] - e2[1])
    if res != 0:
        return res
    else:
        a = e1[0] + e2[0]
        b = e2[0] + e1[0]
        if a > b:
            return 1
        elif a == b:
            return 0
        else:
            return -1

In [8]:
# TF-IDF类
class TfIdf(object):
    # 四个参数分别是：训练好的idf字典，默认idf值，处理后的待提取文本，关键词数量
    def __init__(self, idf_dic, default_idf, word_list, keyword_num):
        self.word_list = word_list
        self.idf_dic, self.default_idf = idf_dic, default_idf
        self.tf_dic = self.get_tf_dic()
        self.keyword_num = keyword_num

    # 统计tf值
    def get_tf_dic(self):
        tf_dic = {}
        for word in self.word_list:
            tf_dic[word] = tf_dic.get(word, 0.0) + 1.0

        tt_count = len(self.word_list)
        for k, v in tf_dic.items():
            tf_dic[k] = float(v) / tt_count

        return tf_dic

    # 按公式计算tf-idf
    def get_tfidf(self):
        tfidf_dic = {}
        for word in self.word_list:
            idf = self.idf_dic.get(word, self.default_idf)
            tf = self.tf_dic.get(word, 0)

            tfidf = tf * idf
            tfidf_dic[word] = tfidf

        tfidf_dic.items()
        # 根据tf-idf排序，去排名前keyword_num的词作为关键词
        for k, v in sorted(tfidf_dic.items(), key=functools.cmp_to_key(cmp), reverse=True)[:self.keyword_num]:
            print(k + "/ ", end='')
        print()

In [9]:
# 统计文档集docs的idf字典
idf_dic , default_idf = train_idf(docs_list)
list(idf_dic.items())[:5]

[('来看', 4.328537949722896),
 ('前提', 5.02168513028284),
 ('改善', 4.7340030578310595),
 ('观点', 5.427150238391005),
 ('投入', 4.328537949722896)]

In [10]:
# 单个文档doc
doc = data[0]['content'].strip()
doc

'在12.5元至13元区间，还是值得投入长持。有一些前提一定要明确的，一是浦发的情况已经改善，还会继续完备经营管理向好。二来是市场中的浦发走向已转向右侧交易的方向，与几年前已不一样。从外资进来增持，从股市机制变化，从完备做空机制等等都有利于资金进一步加入浦发阵营。浦发今年的回归合理估值慢于好几个银行，这种情况将于现在起的一段时间会赶上。浦发的第三季业绩已公布。现阶段进入浦发，已没有几年前强弓之末时的失落。好好珍惜这时光。（从浦发明确15.05元已是最低可接受的转股价来看，向上的空间还是不少，虽然不是即时而要一些时间，但浦发已给你确立明确的信号。）2020年上证到2600点以下的观点不变，并不影响我对浦发的驻守。'

In [14]:
# 对文档doc进行分词过滤
split_words = list(jieba.cut(doc))
doc_list = [word for word in split_words if word not in stop_words and word!=" "]
doc_list[:10]

['12.5', '元至', '13', '元', '区间', '值得', '投入', '长持', '前提', '一定']

## 实验结果

In [16]:
# 抽取文档前十doc的高频词
tfidf = TfIdf(idf_dic, default_idf, doc_list, 10)
tfidf.get_tfidf()

浦发/ 明确/ 完备/ 机制/ 几年/ 情况/ 阵营/ 长持/ 还会/ 转向/ 


## TextRank方法

In [17]:
# 主题模型
class TopicModel(object):
    # 三个传入参数：处理后的数据集，关键词数量，具体模型（LSI、LDA），主题数量
    def __init__(self, doc_list, keyword_num, model='LSI', num_topics=4):
        # 使用gensim的接口，将文本转为向量化表示
        # 先构建词空间
        self.dictionary = corpora.Dictionary(doc_list)
        # 使用BOW模型向量化
        corpus = [self.dictionary.doc2bow(doc) for doc in doc_list]
        # 对每个词，根据tf-idf进行加权，得到加权后的向量表示
        self.tfidf_model = models.TfidfModel(corpus)
        self.corpus_tfidf = self.tfidf_model[corpus]

        self.keyword_num = keyword_num
        self.num_topics = num_topics
        # 选择加载的模型
        if model == 'LSI':
            self.model = self.train_lsi()
        else:
            self.model = self.train_lda()

        # 得到数据集的主题-词分布
        word_dic = self.word_dictionary(doc_list)
        self.wordtopic_dic = self.get_wordtopic(word_dic)

    def train_lsi(self):
        lsi = models.LsiModel(self.corpus_tfidf, id2word=self.dictionary, num_topics=self.num_topics)
        return lsi

    def train_lda(self):
        lda = models.LdaModel(self.corpus_tfidf, id2word=self.dictionary, num_topics=self.num_topics)
        return lda

    def get_wordtopic(self, word_dic):
        wordtopic_dic = {}

        for word in word_dic:
            single_list = [word]
            wordcorpus = self.tfidf_model[self.dictionary.doc2bow(single_list)]
            wordtopic = self.model[wordcorpus]
            wordtopic_dic[word] = wordtopic
        return wordtopic_dic

    # 计算词的分布和文档的分布的相似度，取相似度最高的keyword_num个词作为关键词
    def get_simword(self, word_list):
        sentcorpus = self.tfidf_model[self.dictionary.doc2bow(word_list)]
        senttopic = self.model[sentcorpus]

        # 余弦相似度计算
        def calsim(l1, l2):
            a, b, c = 0.0, 0.0, 0.0
            for t1, t2 in zip(l1, l2):
                x1 = t1[1]
                x2 = t2[1]
                a += x1 * x1
                b += x1 * x1
                c += x2 * x2
            sim = a / math.sqrt(b * c) if not (b * c) == 0.0 else 0.0
            return sim

        # 计算输入文本和每个词的主题分布相似度
        sim_dic = {}
        for k, v in self.wordtopic_dic.items():
            if k not in word_list:
                continue
            sim = calsim(v, senttopic)
            sim_dic[k] = sim

        for k, v in sorted(sim_dic.items(), key=functools.cmp_to_key(cmp), reverse=True)[:self.keyword_num]:
            print(k + "/ ", end='')
        print()

    # 词空间构建方法和向量化方法，在没有gensim接口时的一般处理方法
    def word_dictionary(self, doc_list):
        dictionary = []
        for doc in doc_list:
            dictionary.extend(doc)

        dictionary = list(set(dictionary))

        return dictionary

    def doc2bowvec(self, word_list):
        vec_list = [1 if word in word_list else 0 for word in self.dictionary]
        return vec_list

In [18]:
def textrank_extract(text, pos=False, keyword_num=10):
    textrank = analyse.textrank
    keywords = textrank(text, keyword_num)
    # 输出抽取出的关键词
    for keyword in keywords:
        print(keyword + "/ ", end='')
    print()

## 实验结果

In [19]:
textrank_extract(doc)

机制/ 完备/ 不变/ 长持/ 走向/ 公布/ 没有/ 转向/ 进入/ 投入/ 


# 自动摘要

## 数据准备

In [21]:
# 五个句子组成的文本
text = """Automatic summarization is the process of reducing a text document with a \
computer program in order to create a summary that retains the most important points \
of the original document. As the problem of information overload has grown, and as \
the quantity of data has increased, so has interest in automatic summarization. \
Technologies that can make a coherent summary take into account variables such as \
length, writing style and syntax. An example of the use of summarization technology \
is search engines such as Google. Document summarization is another."""

## TextRank方法

In [24]:
# 选其中最重要的一个句子
summarizer.summarize(text)

'Automatic summarization is the process of reducing a text document with a computer program in order to create a summary that retains the most important points of the original document.'